# Quickhull
### method of computing the convex hull of a finite set of points in 2-dimensional space


In [ ]:
from ipynb.fs.full.drawing_tool import *
%matplotlib notebook
Tolerance = 10e-12

## 1. Auxiliary functions:

Finding the farthest point from the segment with ends x and y:

In [ ]:
def find_distance(x, y, z):   # distance between point (z) and segment (xy)
    a = x[1] - y[1]
    b = y[0] - x[0]
    c = x[0] * y[1] - y[0] * x[1]
    return abs(a * z[0] + b * z[1] + c) / np.sqrt(a * a + b * b)

In [ ]:
def find_farthest_point(x, y, points):
    the_farthest = None
    the_farthest_distance = -float('inf')
    
    for point in points:
        tmp = find_distance(x, y, point)
        if the_farthest_distance < tmp:
            the_farthest = point
            the_farthest_distance = tmp

    return the_farthest

Function returns 1 when point (c) is to the left of the segment (xy) and 0 otherwise:

In [ ]:
def orient(a, b, c):
    det_ = a[0] * b[1] + b[0] * c[1] + c[0] * a[1] - c[0] * b[1] - b[0] * a[1] - a[0] * c[1]
    
    if det_ > Tolerance:
        return 1
    return 0

Function returns only points that are to the left of the segment (xy):

In [ ]:
def split_points(x, y, points):
    
    new = []
    for point in points:
        o = orient(x, y, point)
        if o == 1:
            new.append(point)
            
    return new

##### Functions needed to visualisation:

In [ ]:
def find_point(a, b, c, d):  # function finds the intersection of two lines
    x = (d - b) / (a - c)
    y = (a * d - b * c) / (a - c)
    return (x, y)

Function finds the point of intersection of the line perpendicular to the xy line passing through the point z.

For visualization - it is not needed for the algorithm itself:

In [ ]:
def intersection_point_vis(x, y, z):
    a = (x[1] - y[1]) / (x[0] - y[0])
    b = x[1] - a * x[0]
    c = -1 / a
    d = z[1] - c * z[0]
    point = find_point(a, b, c, d)
    return point

## 2. Main algorithm:

In [ ]:
def quickhull_help(a, b, points):
    def add_scene():   # visualization
        nonlocal convex_hull, points, farthest_point, a, b
        global scenes, data, hull, points_hull
        point = intersection_point_vis(a, b, farthest_point)
        n = len(points_hull)
        points_hull2 = []
        for i in range(n):
            if a == points_hull[i]:
                points_hull2.append(a)
                points_hull2.append(farthest_point)
            else:
                points_hull2.append(points_hull[i])
        points_hull = points_hull2.copy()
        n = len(points_hull)
        lines = [(points_hull[i], points_hull[(i + 1) % n]) for i in range(n)]
        lines2 = [(point, farthest_point), (a, b)]
        scenes.append(Scene([PointsCollection(data, color='skyblue'),
                             PointsCollection(points_hull.copy(), color='deeppink')],
                            [LinesCollection(lines2, color='pink'),
                             LinesCollection(lines, color='deeppink')]))
        
    convex_hull = []
    
    if len(points) == 0 or a is None or b is None:
        return convex_hull
    
    farthest_point = find_farthest_point(a, b, points)
    points.remove(farthest_point)
    
    add_scene()
    
    new1 = split_points(a, farthest_point, points)
    new2 = split_points(farthest_point, b, points)
    
    convex_hull += quickhull_help(a, farthest_point, new1)
    convex_hull += [farthest_point]
    convex_hull += quickhull_help(farthest_point, b, new2)

    return convex_hull

In [ ]:
def quickhull(points):
    global hull
    
    if len(points) <= 2:
        return points
    
    sorted_points = sorted(points, key = lambda p: (p[0], p[1]))
    
    a = sorted_points[0]
    b = sorted_points[-1]
    sorted_points.pop(0)
    sorted_points.pop()
    points_hull.append(a)
    points_hull.append(b)
    
    above = split_points(a, b, sorted_points)
    below = split_points(b, a, sorted_points)
    
    scenes.append(Scene([PointsCollection(points_hull, color='deeppink'),
                        PointsCollection(data, color='skyblue')],
                    [LinesCollection([(a, b)], color='deeppink')]))
    
    hull += [a]
    hull += quickhull_help(a, b, above)
    hull += [b]
    hull += quickhull_help(b, a, below)
    
    n = len(hull)
    lines = [(hull[i], hull[(i + 1) % n]) for i in range(n)]
    scenes.append(Scene([PointsCollection(data, color='skyblue'),
                         PointsCollection(hull, color='deeppink')],
                    [LinesCollection(lines, color='deeppink')]))
    return hull

##### Loading a set of points from a json file:

In [ ]:
with open('points.json', 'r') as file:
    data = js.loads(file.read())

In [ ]:
scenes = []
hull = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = quickhull(data)
plot = Plot(scenes=scenes)
plot.draw()

## 3. Points generators:

Returns set of points with vertexes of the rectangle and randomly placed points on the sides and diagonals of this rectangle:

In [ ]:
def on_retangle(n_sides, n_diagonals, vertexes):

    points = [vertexes[0], vertexes[1], vertexes[2], vertexes[3]]
    
    for i in range(n_sides):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = random.uniform(vertexes[0][1], vertexes[1][1])
        points.append((x1, y1))
        x2 = random.uniform(vertexes[0][0], vertexes[3][0])
        y2 = random.uniform(vertexes[0][1], vertexes[3][1])
        points.append((x2, y2))
        
    for i in range(n_diagonals):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = x1 + vertexes[0][1]
        points.append((x1, y1))
        x2 = random.uniform(vertexes[3][0], vertexes[2][0])
        y2 = -x2 + vertexes[3][1]
        points.append((x2, y2))
        
    return points

Returns set of points randomly placed on the circle:

In [ ]:
def on_circle(n, s, r):
    
    points = []
    for i in range(n):
        a = random.uniform(0, 2*np.pi)
        x = np.cos(a) * (r ** 2) + s[0]
        y = np.sin(a) * (r ** 2) + s[1]
        points.append((x, y))
        
    return points

Returns set of points randomly placed inside the rectangle:

In [1]:
def randoms(n, p_x, p_y):
    
    points = []
    for _ in range(n):
        x = random.uniform(p_x[0], p_x[1])
        y = random.uniform(p_y[0], p_y[1])
        points.append((x, y))

    return points

## 4. For saving points entered with the mouse:

In [ ]:
def save_plot(plot, name):
    
    points = []
    for i in range(len(plot.get_added_points())):
        for point in plot.get_added_points()[i].points:
            points.append(point)

    with open(f'{name}.json', 'w') as file:
       file.write(js.dumps(points))

In [ ]:
plot = Plot(scenes=[Scene()])
plot.draw()

In [ ]:
save_plot(plot, "ccc")

## 5. Examples:

In [ ]:
data = on_retangle(50, 50, [(0, 0), (1, 0), (1, 1), (0, 1)])
scenes = []
hull = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = quickhull(data)
plot = Plot(scenes=scenes)
plot.draw()

In [ ]:
data = on_circle(100, (0, 0), 1)
scenes = []
hull = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = quickhull(data)
plot = Plot(scenes=scenes)
plot.draw()

In [ ]:
data = randoms(100, [0, 100], [0, 100])
scenes = []
hull = []
points_hull = []
scenes.append(Scene([PointsCollection(data, color='skyblue')]))
hull = quickhull(data)
plot = Plot(scenes=scenes)
plot.draw()